# Day 3 - Lab 1: AI-Driven Backend Development

**Objective:** Generate a complete FastAPI backend application, including Pydantic and SQLAlchemy models, and then perform the critical engineering task of integrating the generated code with the live SQLite database created on Day 2.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 3! With our requirements and architecture defined, it's time to write code. In this lab, you will act as a senior developer guiding an AI co-pilot. Your task is to generate the full backend API for the Onboarding Tool. This involves not just generating code, but also connecting it to the live database we created yesterday, moving from a prototype to a functional, data-driven application.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We'll set up our environment and load the `schema.sql` artifact from Day 2. This SQL file contains the `CREATE TABLE` statements that define our database structure, which is the perfect context to provide the LLM for code generation.

**Model Selection:**
For code generation, models specifically fine-tuned for coding are ideal. `gpt-4.1`, `o3`, or `codex-mini` are excellent choices. Experiment to see which one gives you the cleanest code.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read the SQL schema.
- `save_artifact()`: To save the generated Python code.
- `clean_llm_output()`: To remove markdown fences from the generated code.

In [ ]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output,prompt_enhancer

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

# Load the SQL schema from Day 2
sql_schema = load_artifact("artifacts/waffle_schema.sql")
if not sql_schema:
    print("Warning: Could not load waffle_schema.sql. Lab may not function correctly.")

2025-10-02 12:49:19,434 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


## Step 2: The Challenges

Follow the challenges below to build and connect your API.

### Challenge 1 (Foundational): Generating Code with In-Memory Logic

**Task:** Generate all the necessary Python code for a FastAPI application, but with simple in-memory data storage for now. This allows us to generate and validate the code's structure before adding database complexity.

**Instructions:**
1.  Create a detailed prompt that asks the LLM to act as a senior Python developer.
2.  Provide the `sql_schema` as context.
3.  Instruct the LLM to generate three key components:
    * **Pydantic Models:** For API data validation (request/response bodies).
    * **FastAPI Endpoints:** Full CRUD (Create, Read, Update, Delete) endpoints for the `users` table.
    * **In-Memory Database:** A simple Python list to act as a temporary, fake database.
4.  The final output should be a single Python script for a `main_in_memory.py` file.
5.  Save the generated code to `app/main_in_memory.py`.

In [ ]:
# TODO: Write a prompt to generate a complete FastAPI application with in-memory data storage.
in_memory_api_prompt = f"""
As a senior Python developer, using {sql_schema} as the database schema generate the followings:\
- Pydantic models for API data validation : provide request and response bodies\
- FastAPI endpoints: Create a Full CRUD (Create, Read, Update, Delete) endpoints for the users table\
- In-memory Database: A simple python list to act as a temporary database\

Output:
- a single python script

Context:
Schema: {sql_schema}
"""
in_memory_api_prompt = prompt_enhancer(in_memory_api_prompt)
print(in_memory_api_prompt)
print("--- Generating FastAPI app with in-memory database ---")
if sql_schema:
    generated_api_code = get_completion(in_memory_api_prompt, client, model_name, api_provider)
    cleaned_code = clean_llm_output(generated_api_code, language='python')
    print(cleaned_code)
    save_artifact(cleaned_code, "app/waffle_main_in_memory.py")
else:
    print("Skipping API generation because schema is missing.")

2025-10-02 12:00:29,303 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


<prompt>
  <persona>
    You are a Senior Python Backend Engineer who specializes in FastAPI, Pydantic, and rapid prototyping. Leverage best-practice patterns (type hints, dependency injection, clear separation of concerns, and exhaustive validation) while writing clean, idiomatic Python 3.11 code.
  </persona>

  <context>
    Database schema to model (note the NOT NULL, UNIQUE, CHECK and FK constraints):

    CREATE TABLE users (
        user_id   INTEGER PRIMARY KEY AUTOINCREMENT,
        email      TEXT NOT NULL UNIQUE,
        first_name TEXT NOT NULL,
        last_name  TEXT NOT NULL,
        start_date DATE NOT NULL,
        role_id    INTEGER NOT NULL,
        CHECK (length(email)      > 5),
        CHECK (length(first_name) > 1),
        CHECK (length(last_name)  > 1),
        FOREIGN KEY (role_id) REFERENCES user_roles(role_id)
    );

    CREATE TABLE user_roles (
        role_id   INTEGER PRIMARY KEY AUTOINCREMENT,
        role_name TEXT NOT NULL UNIQUE,
        CHECK (leng

### Challenge 2 (Intermediate): Generating Database Models and Session Code

**Task:** Now, generate the specific SQLAlchemy code required to connect our application to the live `onboarding.db` SQLite database.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `sql_schema` as context again.
3.  Instruct the LLM to generate two separate pieces of code:
    * **SQLAlchemy Models:** Python classes that map to your database tables.
    * **Database Session Management:** The boilerplate code to create a database engine, session maker, and a dependency function (`get_db`) for use in FastAPI.
4.  The output should be two distinct, well-commented Python code blocks. We will integrate these manually in the next step.

In [3]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
db_code_prompt = f"""
# As an expert Python developer, your task is to generate 2 separate pieces of code:\

Task:

1. Create python classes that maps to the database tables {sql_schema}
2. the boilerplate code to create a database engine, session maker, and a dependency function like get_db() for use in FastAPI. Include db.append() methods as well

Conditions:
The output should be 2 distinct, will commented python code snippets
"""

db_code_prompt = prompt_enhancer(db_code_prompt)
print(db_code_prompt)

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
    save_artifact(generated_db_code, "app/capstone_helper.py")
else:
    print("Skipping DB code generation because schema is missing.")

2025-10-02 12:01:11,192 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=o3 latency_ms=None artifacts_path=None


<prompt>
    <persona>
        You are an expert Python backend engineer with 10+ years of experience designing production-grade
        SQLAlchemy models and FastAPI database layers.
    </persona>

    <context>
        Database schema to be replicated in SQLAlchemy ORM:

        CREATE TABLE users (
            user_id INTEGER PRIMARY KEY AUTOINCREMENT,
            email TEXT NOT NULL UNIQUE,
            first_name TEXT NOT NULL,
            last_name TEXT NOT NULL,
            start_date DATE NOT NULL,
            role_id INTEGER NOT NULL,
            CHECK (length(email) > 5),
            CHECK (length(first_name) > 1),
            CHECK (length(last_name) > 1),
            FOREIGN KEY (role_id) REFERENCES user_roles(role_id)
        );

        CREATE TABLE user_roles (
            role_id INTEGER PRIMARY KEY AUTOINCREMENT,
            role_name TEXT NOT NULL UNIQUE,
            CHECK (length(role_name) > 1)
        );
    </context>

    <instructions>
        1. Generate TWO di

### Challenge 3 (Advanced): Integrating Live Database Logic

**Task:** This is the most critical engineering step of the lab. You will manually integrate the generated database code into the FastAPI application, replacing the in-memory logic with live database operations.

**Instructions:**
This task represents a significant jump in complexity. Follow these steps carefully in your IDE (like VS Code):

1.  Create a new, empty file named `app/main.py`.
2.  **First, copy the Pydantic models and the `app = FastAPI()` line** from your `app/main_in_memory.py` file and paste them into `app/main.py`.
3.  **Next, paste the SQLAlchemy model classes and the `get_db` dependency function** you generated in Challenge 2 into your new `app/main.py`.
4.  **Now, let's refactor the `POST /users/` endpoint.** Copy the endpoint function from the in-memory file, but replace the in-memory logic (e.g., `db.append()`) with the correct SQLAlchemy session calls: `db.add(db_user)`, `db.commit()`, and `db.refresh(db_user)`.
5.  Repeat this refactoring process for the other endpoints (GET, PUT, DELETE), replacing list manipulations with the appropriate SQLAlchemy `db.query()` methods.

This task requires you to act as the senior developer, stitching together the AI-generated components into a functional, cohesive whole. You may need to ask the LLM follow-up questions like, "How do I write a SQLAlchemy query to find a user by ID?"

In [ ]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
main_py = load_artifact("app/waffle_main.py")
db_code_prompt = f"""
# As an expert Python developer, your task is to generate python code snippet for GET, PUT and DELETE endpoints in the {main_py}
Use create_user function as a reference. 

Context : {sql_schema}

Expected Output: 
I would like to see GET, PUT, and DELETE functions generated just like POST function create_user in {main_py}
"""

db_code_prompt = prompt_enhancer(db_code_prompt)
print(db_code_prompt)

print("--- Generating SQLAlchemy Models and Session Code ---")
if sql_schema:
    generated_db_code = get_completion(db_code_prompt, client, model_name, api_provider)
    print("\n--- Generated Database Code ---")
    print(generated_db_code)
else:
    print("Skipping DB code generation because schema is missing.")

In [1]:
# TODO: Write a prompt to generate SQLAlchemy models and the database session/dependency code.
main_py = load_artifact("app/waffle_main.py")
code_prompt = f"""
# As an expert Python developer, your task is to validate the python code snippet in the {main_py} with test cases for GET, PUT and DELETE endpoints \
and ensure all the functions are working as expected\


Expected Output: 
I would like to see all the functions working and the testing being done to validate the functions. Also provide the API end points.
"""

code_prompt = prompt_enhancer(code_prompt)
print(get_completion(code_prompt, client, model_name, api_provider))

NameError: name 'load_artifact' is not defined

## Lab Conclusion

Congratulations! You have successfully generated and assembled a complete, database-connected backend API. You used an LLM to generate the boilerplate for both the API endpoints and the database models, and then performed the crucial engineering task of integrating them. You now have a working `main.py` file in your `app` directory that can create, read, update, and delete data in a live database. In the next lab, we will write a comprehensive test suite for this API.

> **Key Takeaway:** AI excels at generating boilerplate code (like models and endpoint structures), but the developer's critical role is in the final integration and wiring of these components into a coherent, working system.